In [ ]:
# EVALUACIÓN FINAL: ANÁLISIS DEL IMPACTO DEL CAMBIO CLIMÁTICO EN LA PRODUCCIÓN AGRÍCOLA
# Eres un analista de datos en un centro de investigación sobre cambio climático y seguridad alimentaria. Te han encomendado evaluar cómo factores 
# climáticos afectan la producción agrícola en distintos países. Para ello, aplicarás modelos de aprendizaje supervisado para predecir la producción de 
# alimentos y clasificar los países según su vulnerabilidad.
# OBJETIVOS DE LA EVALUACIÓN
# • Aplicar técnicas de regresión para predecir la producción agrícola en función de variables climáticas.
# • Utilizar algoritmos de clasificación para categorizar los países según el impacto del cambio climático.
# • Evaluar el desempeño de los modelos mediante métricas adecuadas.
# • Aplicar preprocesamiento de datos para mejorar la calidad del análisis.
# • Optimizar los modelos a través de técnicas de ajuste de hiperparámetros.


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score


# INSTRUCCIONES
# 1. Carga y exploración de datos 
# • Carga el dataset proporcionado, que contiene información sobre temperatura media, cambio en las precipitaciones, frecuencia de sequías y producción agrícola en distintos países.
# • Analiza la distribución de las variables y detecta posibles valores atípicos o tendencias.

df = pd.read_csv('cambio_climatico_agricultura.csv')

print('Descripción')
display(df.describe())
print('Primeras 5 filas:')
display(df.head())
 
# Revisar valores faltantes
print('Valores faltantes por columna:')
print(df.isnull().sum())



Descripción


,Temperatura_promedio,Cambio_lluvias,Frecuencia_sequías,Producción_alimentos
count,23.000000,23.000000,23.000000,2.300000e+01
mean,16.291304,-7.782609,4.347826,7.030435e+05
std,6.692256,4.055933,2.604344,2.410665e+05
min,4.600000,-15.000000,1.000000,3.000000e+05
25%,12.150000,-11.000000,2.000000,5.250000e+05
50%,15.400000,-8.000000,4.000000,7.000000e+05
75%,21.400000,-4.500000,6.500000,8.950000e+05
max,28.000000,-1.000000,9.000000,1.200000e+06


Primeras 5 filas:


,País,Temperatura_promedio,Cambio_lluvias,Frecuencia_sequías,Producción_alimentos
0,Alemania,10.5,-5,2,950000
1,Argentina,17.8,-10,5,650000
2,Australia,22.3,-15,8,400000
3,Brasil,25.1,-8,6,900000
4,Canadá,6.2,-2,1,850000


Valores faltantes por columna:
País                    0
Temperatura_promedio    0
Cambio_lluvias          0
Frecuencia_sequías      0
Producción_alimentos    0
dtype: int64


In [ ]:
# 2. Preprocesamiento y escalamiento de datos 
# • Aplica técnicas de normalización o estandarización a las variables numéricas.
# • Codifica correctamente cualquier variable categórica si fuera necesario.
# • Divide los datos en conjunto de entrenamiento y prueba (80%-20%).
'''
x = df.drop('País', axis=1)
y = df['País']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=42) 
'''


# a. Detectar la columna de producción
prod_col = None
for candidate in ["produccion", "producción", "production", 
                  "agricultural_production", "yield", 
                  "produccion_agricola", "producción_alimentos"]:
    if candidate in df.columns.str.lower():
        prod_col = df.columns[df.columns.str.lower() == candidate][0]
        break
if prod_col is None:
    candidates = [c for c in df.columns if "prod" in c.lower() or "yield" in c.lower()]
    prod_col = candidates[0] if candidates else df.columns[-1]  # fallback

print(f"Columna de producción detectada: {prod_col}")

# b. Separar columnas numéricas y categóricas
num_cols = df.select_dtypes(include="number").columns.tolist()
cat_cols = df.select_dtypes(exclude="number").columns.tolist()

# c. One-Hot Encoding para categóricas
df_prep = df.copy()
if cat_cols:
    df_prep = pd.get_dummies(df_prep, columns=cat_cols, drop_first=True)

# d. Escalamiento
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()

df_standardized = df_prep.copy()
df_normalized = df_prep.copy()

df_standardized[num_cols] = scaler_standard.fit_transform(df_standardized[num_cols])
df_normalized[num_cols] = scaler_minmax.fit_transform(df_normalized[num_cols])

print("\nEjemplo de datos estandarizados:")
print(df_standardized.head())

print("\nEjemplo de datos normalizados:")
print(df_normalized.head())

# 5. División en entrenamiento y prueba (80%-20%) usando estandarizados
X = df_standardized.drop(columns=[prod_col])
y = df_standardized[prod_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nDimensiones de los conjuntos:")
print(f"Entrenamiento -> X: {X_train.shape}, y: {y_train.shape}")
print(f"Prueba        -> X: {X_test.shape}, y: {y_test.shape}")




Columna de producción detectada: Producción_alimentos

Ejemplo de datos estandarizados:
   Temperatura_promedio  Cambio_lluvias  Frecuencia_sequías  \
0             -0.884823        0.701478           -0.921765   
1              0.230506       -0.558990            0.256046   
2              0.918037       -1.819458            1.433856   
3              1.345835       -0.054803            0.648649   
4             -1.541798        1.457759           -1.314368   

   Producción_alimentos  País_Argentina  País_Australia  País_Brasil  \
0              1.047457           False           False        False   
1             -0.224982            True           False        False   
2             -1.285348           False            True        False   
3              0.835384           False           False         True   
4              0.623311           False           False        False   

   País_Canadá  País_Chile  País_China  ...  País_Italia  País_Japón  \
0        False       False  

In [ ]:

# 3. Aplicación de modelos de aprendizaje supervisado 
# • Regresión:
# o Entrena un modelo de regresión lineal para predecir la producción de alimentos.
# o Evalúa el modelo usando métricas como MAE, MSE y R2.
# o Compara con otros modelos de regresión (árbol de decisión, random forest).

# • Clasificación:
# o Crea una nueva variable categórica que clasifique los países en "Bajo", "Medio" y "Alto" impacto climático en la producción agrícola.
# o Entrena modelos de clasificación como K-Nearest Neighbors, Árbol de Decisión y Support Vector Machine.
# o Evalúa el desempeño usando matriz de confusión, precisión, sensibilidad y curva ROC-AUC.

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# a. Crear y entrenar el modelo
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# b. Predicciones
y_pred_train = reg_model.predict(X_train)
y_pred_test = reg_model.predict(X_test)

# c. Evaluación
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("\n--- Evaluación del modelo ---")
print(f"RMSE Entrenamiento: {rmse_train:.4f}")
print(f"RMSE Prueba       : {rmse_test:.4f}")
print(f"MAE Entrenamiento : {mae_train:.4f}")
print(f"MAE Prueba        : {mae_test:.4f}")
print(f"R² Entrenamiento  : {r2_train:.4f}")
print(f"R² Prueba         : {r2_test:.4f}")

# d. Coeficientes del modelo
coef_df = pd.DataFrame({
    "Variable": X_train.columns,
    "Coeficiente": reg_model.coef_
}).sort_values(by="Coeficiente", ascending=False)

print("\nCoeficientes del modelo")
print(coef_df)




In [ ]:
# 4. Optimización de modelos 
# • Ajusta hiperparámetros utilizando validación cruzada y búsqueda en grilla.
# • Aplica técnicas de regularización y analiza su impacto en los modelos.

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# a. Definir modelos y parámetros
models = {
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42)
}

params = {
    "DecisionTree": {
        "max_depth": [3, 5, 10, None],
        "min_samples_split": [2, 5, 10]
    },
    "RandomForest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 10, None],
        "min_samples_split": [2, 5, 10]
    }
}

# b. Búsqueda en grilla con validación cruzada
best_models = {}
for name, model in models.items():
    grid = GridSearchCV(model, params[name], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_
    print(f"\nMejores parámetros para {name}: {grid.best_params_}")
    y_pred = best_models[name].predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"RMSE en prueba para {name}: {rmse:.4f}")


# c. Evaluación de los mejores modelos
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n--- Evaluación del modelo optimizado: {name} ---")
    print(f"RMSE Prueba: {rmse:.4f}")
    print(f"MAE Prueba : {mae:.4f}")
    print(f"R² Prueba  : {r2:.4f}")


# 5. Análisis de resultados y conclusiones 
# • Compara los modelos utilizados y justifica cuál ofrece mejores resultados para la predicción y clasificación.
# • Relaciona los hallazgos con posibles implicaciones en la seguridad alimentaria global.

''' 
En este análisis, hemos aplicado varios modelos de regresión para predecir la producción agrícola en función de variables climáticas. 
Hemos evaluado modelos como la regresión lineal, árbol de decisión y random forest, optimizando sus hiperparámetros mediante búsqueda en grilla y 
validación cruzada.
Los resultados indican que el modelo de Random Forest ofrece el mejor desempeño en términos de RMSE y R² en el conjunto de prueba, lo que sugiere que es 
capaz de capturar mejor las complejidades y no linealidades presentes en los datos climáticos y su impacto en la producción agrícola.  
Además, los coeficientes del modelo de regresión lineal proporcionan información valiosa sobre la influencia relativa de cada variable climática en la 
producción agrícola, lo que puede ser útil para diseñar estrategias de mitigación y adaptación al cambio climático.  Estos hallazgos tienen importantes 
implicaciones para la seguridad alimentaria global, ya que permiten identificar los factores climáticos más críticos que afectan la producción agrícola y, 
por ende, la disponibilidad de alimentos. Esto puede guiar políticas y acciones para mejorar la resiliencia de los sistemas agrícolas frente al cambio climático, 
asegurando un suministro alimentario sostenible en el futuro.   

'''




